# LangChain 및 필요한 라이브러리 설치

In [ ]:
!pip install langchain unstructured[all-docs] pydantic lxml langchain_openai langchain-community chromadb

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (210 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123636 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install pytesseract

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/miraeasset/code

/content/drive/.shortcut-targets-by-id/1JAA9klGBKIyYF5W08h86IbqIHWhkgVTj/miraeasset/code


In [ ]:
import os
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain, TransformChain, LLMChain

import base64
import json
import http.client
import time

import concurrent.futures
from embedding import HyperClovaEmbedding, HyperClovaEmbeddings # custom: embedding.py

from langchain.embeddings.base import Embeddings
from langchain.prompts import PromptTemplate

import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder
from operator import itemgetter

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

import pickle
import requests

# Data Loading

- PDF 표(Tables)와 텍스트(text) 분할
- Unstructured의 partition_pdf를 사용하여 레이아웃 모델을 이용해 PDF 문서를 세분화

In [ ]:
path = '/content/drive/MyDrive/miraeasset/data/'

In [ ]:
# Get elements
raw_pdf_elements = partition_pdf(
    filename=path + "20240725_SK하이닉스.pdf", # miraeasset_report.pdf"

    extract_images_in_pdf=True, # PDF에서 이미지 블록을 추출

    infer_table_structure=True, # 레이아웃 모델을 사용하여 테이블을 감지하고 구조화

    chunking_strategy="by_title", #  문서 제목이나 섹션을 기준으로 텍스트를 나눔

    max_characters=4000, # 청크당 문자 수에 대한 엄격한 제한을 설정
    new_after_n_chars=3800, # 3800자 이후에 새 청크를 시작하려고 시도
    combine_text_under_n_chars=2000, # 2000자 미만인 경우 더 작은 텍스트 블록을 더 큰 덩어리로 결합
    image_output_dir_path=path, # 추출된 이미지를 저장할 디렉터리 경로
)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 14,
 "<class 'unstructured.documents.elements.Table'>": 8}

In [ ]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

8
14


In [ ]:
table_elements[1] # 확인용

Element(type='table', text='1Q23 2Q23 DRAM Bit shipments (Bil Gb) 12,088 16,440 Bit growth -18.0 36.0 ASP ($/Gb) 0.19 0.21 ASP change -16.4 9.4 NAND Bit shipments (Bil GB) 27,344 40,469 Bit growth -16.0 48.0 ASP ($/GB) 0.05 0.04 ASP change -10.2 -11.7 3Q23 19,892 21.0 0.23 11.1 42,897 6.0 0.04 -1.5 4Q23 20,290 2.0 0.27 17.7 42,039 -2.0 0.06 40.8 1Q24 2Q24 17,044 20,623 -16.0 21.0 0.33 0.38 22.2 14.5 42,039 40,778 0.0 -3.0 0.08 0.09 32.0 16.9 3Q24F 21,378 3.7 0.43 12.5 38,739 -5.0 0.10 7.0 4Q24F 22,688 6.1 0.47 8.6 42,613 10.0 0.10 3.0 2022 2023 2024F 2025F 59,389 68,711 81,733 97,353 1.6 15.7 19.0 19.1 0.37 0.23 0.41 0.51 -18.7 -37.9 76.3 25.4 126,757 152,749 164,169 187,469 47.6 20.5 7.5 14.2 0.09 0.05 0.09 0.11 -15.6 -45.5 91.2 18.1')

In [ ]:
text_elements[1] # 확인용

Element(type='text', text='순이익 (십억원)\n\n2,230\n\n9,112\n\n18,322\n\n31,249\n\nEPS (원)\n\n3,063\n\n12,517\n\n25,168\n\n42,924\n\nROE (%)\n\n3.6\n\n15.6\n\n29.4\n\n36.1\n\nP/E (배)\n\n24.5\n\n8.3\n\n4.9\n\nP/B (배)\n\n0.8\n\n1.8\n\n2.1\n\n1.5\n\n배당수익률 (%)\n\n1.6\n\n0.8\n\n0.6\n\n0.6\n\n주: K-IFRS 연결 기준, 순이익은 지배주주 귀속 순이익 자료: SK하이닉스, 미래에셋증권 리서치센터\n\n2026F\n\n101,855\n\n31,236\n\n30.7\n\n25,229\n\n34,655\n\n22.1\n\n6.0\n\n1.2\n\n0.6\n\nSK하이닉스\n\n2 Mirae Asset Securities Research\n\n2024.7.25\n\n실적 전망 및 밸류에이션\n\n동사에 대한 투자의견 ‘매수’ 유지 및 목표주가를 기존 240,000원에서 260,000원으로\n\n8% 상향한다. 예상보다 강한 메모리 가격 전망에 따라 24년/25년 영업이익 추정치를 각\n\n23.4%, 62.5% 상향했으나, HBM에서의 시장지배력 약화를 고려해 타겟 밸류에이션을\n\nP/B 2.3배에서 2.1배로 하향 적용했다.\n\n2Q24 실적은 매출액 16.4조원(QoQ +32.1%), 영업이익 5.5조원(QoQ +89.5%)을 기록\n\n했다. 메모리 가격 인상(DRAM +15.2, NAND +16.0%)이 지속되는 가운데 HBM3E에 대\n\n한 매출액이 본격 반영되기 시작했고, 환율도 우호적이었다.\n\n내년 시장 상황은 더욱 우호적\n\n25년 DRAM 수요 B/G(Bit growth)는 +23.0%로 전망된다. 경기 회복에 대한 명확한 전\n\n망이 어려운 상황에서 올해 B/G(+17.1%)보다 상승폭이 확대되는 부분이 고무적이다. 수\n\n요

# 1. Multi-vector retriever

multi-vector-retriever를 사용하여 표와 선택적으로 텍스트의 요약(summaries)을 생성

요약과 함께 원본 표 요소(raw tables)도 저장

요약은 검색의 품질을 향상시키는 데 사용되며, 이는 multi-vector retriever 공식문서에서 설명

원본 표는 LLM에 전달되어, LLM이 답변을 생성하는 데 전체 표 맥락을 제공











# Summaries

In [ ]:
class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id, retry_attempts=10, initial_retry_delay=5):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
        self.retry_attempts = retry_attempts
        self.initial_retry_delay = initial_retry_delay

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        retry_delay = self.initial_retry_delay
        for attempt in range(self.retry_attempts):
            conn = http.client.HTTPSConnection(self._host)
            conn.request('POST', '/testapp/v1/api-tools/summarization/v2/177a87b6436e4d22815fded90f5d2dce', json.dumps(completion_request), headers)
            response = conn.getresponse()
            if response.status == 429:
                # 요청이 너무 많음 (Rate limit exceeded)
                print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # 지수 백오프 (Exponential backoff)
                continue
            elif response.status != 200:
                conn.close()
                raise ValueError(f"HTTP error: {response.status} - {response.reason}")
            result = json.loads(response.read().decode(encoding='utf-8'))
            conn.close()
            if result['status']['code'] == '20000':
                return result
            else:
                raise ValueError('Error in API response: ' + result.get('status', {}).get('message', 'Unknown error'))

        raise ValueError('Max retry attempts exceeded')

    def execute(self, completion_request):
        try:
            res = self._send_request(completion_request)
            return res['result']['text']
        except ValueError as e:
            print(f"Error: {e}")
            return 'Error'


In [ ]:
# Define a function to use the HyperCLOVA API for summarization
def hyperclova_summarize(text):
    completion_executor = CompletionExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='your_api_key_here',  # Replace with your actual API key
        api_key_primary_val='your_primary_api_key_here',  # Replace with your actual primary API key
        request_id='your_request_id_here' # Replace with your actual request ID
    )

    request_data = {
        "texts": [text],
        "segMinSize": 300,
        "includeAiFilters": True,
        "autoSentenceSplitter": True,
        "segCount": -1,
        "segMaxSize": 1000
    }

    response_text = completion_executor.execute(request_data)
    return response_text

In [ ]:
# Process texts concurrently and maintain order
def process_texts_concurrently(texts):
    results = [None] * len(texts)  # Placeholder for results
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Create a dictionary to track which future corresponds to which index
        future_to_index = {executor.submit(hyperclova_summarize, text): i for i, text in enumerate(texts)}
        for future in concurrent.futures.as_completed(future_to_index):
            index = future_to_index[future]
            try:
                result = future.result()
                results[index] = result
            except Exception as exc:
                print(f'Text at index {index} generated an exception: {exc}')
                results[index] = None  # Or handle the exception as needed
    return results

In [ ]:
completion_executor = CompletionExecutor(
    host='clovastudio.apigw.ntruss.com',
    api_key='your_api_key_here',  # Replace with your actual API key
    api_key_primary_val='your_primary_api_key_here',  # Replace with your actual primary API key
    request_id='your_request_id_here'  # Replace with your actual request ID
)

## 요약 예시

In [ ]:
request_data = {
    "texts": ["""미래에셋증권은 안정적인 수익률과 개인 맞춤형 자산관리를 제공하기 위해
‘로보어드바이저’ 서비스를 성공적으로 런칭하여 운영하고 있습니다. 이 서비스는
고객의 투자 성향, 가입 시점, 보유 상품 현황 등을 종합적으로 분석하여 최적의
고객 맞춤 포트폴리오를 제공합니다. 또한, 필요에 따라 즉각적인 리밸런싱을 통해
포트폴리오를 조정함으로써 시장 변동에 능동적으로 대응할 수 있습니다.
특히 주목할 만한 성과로, ‘퇴직연금 로보어드바이저’ 서비스는 2023년 7월 1만개
계좌 돌파 이후, 2024년 3월 1만 7천개 이상의 계좌가 활성화되어 있습니다. 또한,
2024년 1월 기준으로 누적 가입 금액 1조 원을 달성하였습니다.
앞으로도 미래에셋증권은 시장의 변동성에 흔들리지 않고 원칙을 지키는 투자를 통해
고객의 안정적인 자산 성장을 추구하기 위해 지속적으로 노력할 것입니다.""",

              """미래에셋증권은 여성들이 안심하고 일에 전념할 수 있는 환경을 만들기 위해 임신
에서 출산, 육아와 관련된 다양한 복지제도를 운영하고 있습니다. 난임 직원에게 법정
기준을 초과하여 유급휴가를 부여하고, 법에서 정한 기간을 초과하는 수준의 출산
휴가사용을 보장하며, 임신중 또는 출산 후 육아휴직 사용과 별도의 육아휴직 급여
지급과 출산 관련 의료비 보조, 출산 경조금 등의 혜택 등을 통해 직원들의 육아
부담을 낮추기 위해 노력하고 있습니다. 특히, 1년 제한이었던 육아휴직 기간을
배우자의 육아휴직 조건없이 1년 6개월까지 늘렸으며, 출산 경조금 확대, 난임
치료의 의료비 보조금 확대도 진행하였습니다. 또한, 여성 직원과 임산부의 건강을
위해 보건, 검진 휴가는 유급으로 부여하며, 임신기 근로시간 단축근무 시행, 태아
검진 시간의 사용 등 가족 친화 근로환경을 보장하기 위한 지속적인 노력을 추구하고
있습니다. 휴직 이외에도 자녀 돌봄을 지원하기 위해 가족 돌봄 휴가를 연간 10일
부여하고 있으며, 임직원이 출산, 육아 부담을 완화하고 일과 가정의 양립을 위해
2011년부터 공동 직장 보육시설(어린이집)을 운영하고 있습니다. 이외에도 센터원
사옥 내에 수유시설 및 여성전용휴게실을 갖추어 출산 후 근무환경 개선을 위해서도
노력하고 있습니다. 육아기에는 임직원의 필요에 따라 근로시간 단축근무 제도를
권장하고 있습니다. 2023년 육아휴직 사용자는 총 202명이며 육아휴직 사용 후
복직률은 97%를 기록하고 있습니다."""],  # Replace with the text you want to summarize
    "segMinSize": 300,
    "includeAiFilters": True,
    "autoSentenceSplitter": True,
    "segCount": -1,
    "segMaxSize": 1000
}

In [ ]:
response_text = completion_executor.execute(request_data)
print(response_text)

- 미래에셋증권은 안정적인 수익률과 개인 맞춤형 자산관리를 제공하기 위해 로보어드바이저 서비스를 성공적으로 런칭하여 운영하고 있음
- 시장의 변동성에 흔들리지 않고 원칙을 지키는 투자를 통해 고객의 안정적인 자산 성장을 추구하기 위해 노력할 것임
- 여성들이 안심하고 일에 전념할 수 있는 환경을 만들기 위해 다양한 복지제도를 운영하고 있음
- 보건, 검진 휴가는 유급으로 부여함
- 임신기 근로시간 단축근무 시행, 태아 검진 시간의 사용 등 가족 친화 근로환경을 보장하기 위한 지속적인 노력을 추구하고 있음
- 임직원이 출산, 육아 부담을 완화하고 일과 가정의 양립을 위해 2011년부터 공동 직장 보육시설(어린이집)을 운영하고 있음
- 육아휴직 사용 후 복직률은 97%를 기록하고 있음


## 요약 실행

In [ ]:
tables = [i.text for i in table_elements]
table_summaries = process_texts_concurrently(tables)

In [ ]:
len(table_summaries)

8

In [ ]:
tables[:2]

['1Q23 2Q23 매출액 5,088 7,306 QoQ/YoY -33.7 43.6 DRAM 2,951 4,530 NAND 1,679 2,265 영업이익 -3,402 -2,882 QoQ/YoY RR RR DRAM -1,476 -91 NAND -1,835 -2,689 영업이익률 -66.9 -39.4 DRAM -50.0 -2.0 NAND -109.3 -118.7 EBITDA 155 612 QoQ/YoY -91.4 295.7 세전이익 -3,525 -3,788 QoQ/YoY RR RR 지배주주 순이익 -2,580 -2,991 QoQ/YoY RR RR 3Q23 9,066 24.1 6,074 2,357 -1,792 RR 607 -2,114 -19.8 10.0 -89.7 1,541 151.9 -2,470 RR -2,184 RR 4Q23 11,306 24.7 7,349 3,279 346 TTB 1,617 -1,169 3.1 22.0 -35.7 3,582 132.5 -1,875 RR -1,357 RR 1Q24 2Q24 12,430 16,423 9.9 32.1 7,582 10,839 4,350 5,091 2,886 5,469 734.0 89.5 2,654 4,661 282 857 23.2 33.3 35.0 43.0 6.5 16.8 6,073 8,591 69.5 41.5 2,373 5,052 TTB 112.9 1,919 0 TTB -100.0 3Q24F 18,424 12.2 12,689 5,194 7,175 31.2 5,822 1,461 38.9 45.9 28.1 10,461 21.8 6,402 26.7 4,607 - 4Q24F 21,062 14.3 14,619 5,884 8,728 21.7 7,167 1,673 41.4 49.0 28.4 12,027 15.0 7,911 23.6 5,693 23.6 2022 2023 2024F 2025F 44,622 32,766 68,338 98,611 3.8 -26.6 108.6 44.3 28,372 20,904 45,729 68,466 14,

In [ ]:
table_summaries[:2]

['- 3분기 매출액과 영업이익을 보고 있음',
 '- DRAM과 NAND의 bit shipments와 bit growth, ASP를 정리하고 있음']

In [ ]:
texts = [i.text for i in text_elements]
text_summaries = process_texts_concurrently(texts)

In [ ]:
len(text_summaries)

14

In [ ]:
texts[6]

'해외사무소 3\n\n* Mirae Asset Securities Holdings(USA) Inc. 제외한 실제 영업법인 기준\n\nAbout Mirae Asset Securities\n\nDaHOWMS 2024 SS\n\n007\n\n경영 철학\n\n경영 철학\n\n고객우선, 미래에셋의 철학과 원칙의 시작입니다.\n\n경영 이념\n\n열린 마음으로 미래를 내다보고 인재를 중시하자\n\n고객 한 분 한 분에게 의미 있는 존재가 되기 위해 미래에셋은 일관된 가치를 추구합니다. 경영이념은 미래에셋이 바탕을 두고 있는 정신이자 나아가고자 하는 변치 않는 지향점입니다.\n\n비전\n\n우리는 글로벌 투자전문그룹으로서 고객의 성공적 자산운용과 평안한 노후를 위해 기여한다\n\n.\n\n.\n\n이머징 마켓 전문가로 시작한 미래에셋은 끊임없는 금융혁신을 통해 글로벌 투자전문그룹으로 자리매김했습니다.\n\nrake,\n\n그리고 그 성장의 방향은 ‘고객의 성공적 자산운용과, 평안한 노후에 기여’하고자 하는 미래에셋의 비전과 언제나 일치하고 있습니다.\n\n미래에셋에는 구성원 개개인이 반드시 마음에 새겨야 할 핵심가치가 존재합니다.\n\nSHAITER|\n\n핵심가치\n\np\n\n미래에셋은 고객을 위해 존재합니다.\n\n고객의 성공이 곧 우리의 성공이라는 신념으로,\n\n고객의 니즈에 맞는 전문화된 금융서비스를 제공하고 고객의\n\n안정적인 수익 창출을 위해 노력하겠습니다.\n\n투자전문그룹으로서 독립성과 경쟁우위를 가집니다. 자산운용, 증권, 보험 등으로 구성된 독립 투자전문 그룹으로서, 원칙을 지키는 투자와 우수한 운용능력을 바탕으로 경쟁우위를 지속하겠습니다.\n\n@\n\nBw\n\n개인을 존중하며 팀플레이에 대한 믿음이 있습니다. 경쟁력 있고 우수한 인재들로 구성되어 있는 미래에셋은 각 개인에 대한 존중과 팀플레이에 대한 믿음을 통해 변화에 앞서가겠습니다.\n\nw\n\n사회적 책임을 인식하고 실천합니다.\n\n이익의 사회환원을 실천하며, 사회에 대한 기여와 봉사를

In [ ]:
text_summaries[6]

'- 미래에셋의 경영 철학과 원칙은 고객 우선, 미래에셋의 철학과 원칙의 시작임\n- 경영 이념은 미래에셋이 바탕을 두고 있는 정신이자 나아가고자 하는 변치 않는 지향점임\n- 비전은 고객의 성공적 자산운용과 평안한 노후에 기여하고자 하는 미래에셋의 비전과 일치함\n- 미래에셋의 핵심가치들을 설명하고 있음\n- 미래에셋증권의 재무적 성과를 설명하고 있음'

# Add to vectorstore


*   InMemoryStore는 원본 텍스트와 표를 저장
*   vectorstore는 임베딩된 요약을 저장

In [ ]:
db_miraeasset = "/content/drive/MyDrive/miraeasset/ChromaDB/DB_miraeasset"

In [ ]:
# HyperClovaEmbedding 인스턴스 생성
hyper_clova = HyperClovaEmbedding(
    host='clovastudio.apigw.ntruss.com',
    api_key='your_api_key_here',
    api_key_primary_val='your_primary_api_key_here',
    request_id='your_request_id_here'
)

In [ ]:
# 임베딩 생성 요청
try:
    text = "input text"
    embedding = hyper_clova.execute(text)
    print(f"Embedding: {embedding}")
except ValueError as e:
    print(e)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

# 재시도 간격을 늘리고, 최대 재시도 횟수를 증가시킵니다.
# 요청 실패 시 백오프(backoff) 전략을 도입하여 재시도 간격을 지수적으로 증가시킵니다.

Embedding: [-0.9326172, 0.69628906, -0.515625, 0.12683105, -0.29736328, -0.55615234, 0.2388916, 0.15234375, 0.40039062, -0.68359375, 0.29541016, -0.047424316, 0.36669922, -0.81689453, -0.2619629, -0.34375, -0.16601562, 0.36743164, 0.17993164, -0.2705078, -0.80078125, -0.24499512, -0.08093262, 0.34033203, 0.47387695, 0.9682617, 0.2866211, -0.42407227, -0.58154297, -0.20178223, 0.87646484, 0.0181427, -0.7939453, -2.0625, 0.31201172, -0.11657715, -0.12573242, -0.3149414, -1.5751953, 0.27807617, 0.087524414, -0.59716797, 0.55615234, -0.022842407, -0.27807617, -0.081970215, 1.0087891, -0.39575195, -0.7050781, -0.5683594, 0.054107666, 0.52685547, 1.8037109, -0.1784668, -0.31225586, -0.5307617, -0.2388916, 0.16149902, -1.6572266, -0.07635498, -0.2788086, -0.54296875, -0.48706055, -0.27929688, -0.21374512, 1.7119141, 0.91308594, 0.13464355, -0.703125, -0.67089844, -0.33496094, 0.7866211, -0.3413086, -0.08496094, -1.2548828, 1.1201172, 0.6401367, -0.40893555, -0.41479492, 0.6435547, 0.6928711, 

In [ ]:
class HyperClovaEmbeddings(Embeddings):
    def __init__(self, hyper_clova_instance):
        self._hyper_clova = hyper_clova_instance

    def embed_documents(self, texts):
        return [self._hyper_clova.execute(text) for text in texts]

    def embed_query(self, text):
        return self._hyper_clova.execute(text)

# HyperClovaEmbedding 인스턴스와 함께 사용하는 임베딩 함수
embedding_func = HyperClovaEmbeddings(hyper_clova)

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=embedding_func)


# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
retriever.vectorstore.similarity_search_with_score("sk하이닉스에 대해 말해줘")

[(Document(metadata={'doc_id': '4ef89bb0-f292-431e-bffc-7c18105b9264'}, page_content='- SK하이닉스의 밸류에이션을 설명하고 있음'),
  393.74163818359375),
 (Document(metadata={'doc_id': '8d94dff9-8509-4f52-8647-d0a0cb621703'}, page_content='- SK하이닉스 주요 제품별 추정치를 보고 있음'),
  539.1354370117188),
 (Document(metadata={'doc_id': '04a4a045-1400-42e1-a7d6-5a585e356d97'}, page_content='- SK하이닉스 수익추정 변경 내역을 보고 있음'),
  547.6119384765625),
 (Document(metadata={'doc_id': 'c51187ba-77ce-4364-89f5-8ba8e84ae01c'}, page_content='- SK하이닉스의 예상 포괄손익계산서와 예상 현금흐름표를 요약함'),
  550.4959106445312)]

# RAG

In [ ]:
from custom_llm import CustomLLM
# CustomLLM 설정
llm = CustomLLM(
    host='https://clovastudio.stream.ntruss.com',
    api_key='your_api_key_here',
    api_key_primary_val='your_primary_api_key_here',
    request_id='your_request_id_here'
)

In [ ]:
# Prompt template
template = """다음 context를 읽고 텍스트 및 테이블을 포함할 수 있는 question에 답하십시오.
context에 없는 내용이면 논리적으로 추론하여 답하십시오.
:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = llm

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## 질문 예시

In [ ]:
chain.invoke("미래에셋 지속가능경영에 대해 말해줘")

'저는 미래에셋증권에서 서비스되는 AI 언어모델로 지속가능경영에 대한 정보를 제공할 수 없습니다. 지속가능경영에 대한 정보는 미래에셋증권의 공식 홈페이지나 관련 뉴스를 참고해주시기 바랍니다.'

In [ ]:
chain.invoke("퇴직연금에 대해 말해줘")

'퇴직연금은 근로자의 노후생활을 보장하기 위해 회사가 근로자에게 지급해야 할 퇴직급여(퇴직금)를 회사가 아닌 금융회사(퇴직연금사업자)에 맡기고, 근로자 퇴직 시 일시금 또는 연금으로 지급하는 제도입니다.\n\n퇴직연금은 확정급여형(DB)과 확정기여형(DC)으로 구분됩니다.\n\n1.확정급여형(DB): 회사가 퇴직연금 재원을 외부 금융회사에 적립하고, 근로자가 퇴직할 때 근속기간과 평균임금에 따라 정해진 금액을 지급하는 방식입니다. 회사가 적립금 운용에 대한 책임을 지며, 근로자는 퇴직급여액이 확정되어 있습니다.\n\n2.확정기여형(DC): 회사가 매년 근로자 연간 임금의 1/12 이상을 부담금으로 납부하고, 근로자가 적립금을 운용하는 방식입니다. 근로자가 적립금 운용에 대한 책임을 지며, 적립금 운용 성과에 따라 퇴직급여액이 변동됩니다.\n\n퇴직연금은 55세 이상이면 수령이 가능하며, 연금으로 수령 시 일시금으로 수령할 때보다 세금 부담이 적습니다.\n\n더 궁금하신 점이나 도움이 필요하신 부분이 있으시면 말씀해주세요. 최선을 다해 도와드리겠습니다.'

In [ ]:

chain.invoke("미래에셋의 경영 철학과 원칙에 대해 말해줘")

'저는 미래에셋증권의 경영 철학과 원칙에 대한 정보를 가지고 있지 않습니다. 미래에셋증권의 공식 웹사이트나 관련 문서를 참고하시면 경영 철학과 원칙에 대해 알 수 있을 것입니다.'

In [ ]:
chain.invoke("sk하이닉스의 주요 제품에 대해 알려줘")

'네. SK하이닉스의 주요 제품은 다음과 같습니다.\n\n- DRAM : 전체 매출의 약 40%를 차지하는 주력 제품으로, 컴퓨터, 서버, 스마트폰 등 다양한 IT 기기에서 사용됩니다.\n\n- NAND Flash : 전체 매출의 약 30%를 차지하는 제품으로, 스마트폰, SSD, USB 등에 사용됩니다.\n\n- MCP(Multi-Chip Package) : DRAM과 NAND Flash를 하나의 패키지에 결합한 제품으로, 스마트폰, 태블릿, 노트북 등에 사용됩니다.\n\n- CMOS 이미지 센서(CIS) : 스마트폰, 노트북, 자동차 등에 사용되는 이미지 센서입니다.\n\n위의 제품들은 SK하이닉스의 주요 제품이며, 시장의 수요와 기술 발전에 따라 다양한 제품을 개발하고 있습니다.'

In [ ]:
chain.invoke("DRAM과 관련하여 성과를 알려줘")

'해당 자료에 따르면, 2024년 DRAM 시장은 전체적으로 안정적인 수요와 공급이 유지될 것으로 예상됩니다. \n\n모바일 DRAM의 수요는 스마트폰의 출하량 증가와 함께 지속적으로 증가할 것으로 예상되며, 서버 DRAM의 수요도 클라우드 서비스의 성장과 함께 꾸준히 증가할 것으로 예상됩니다. 그래픽 DRAM의 경우, 게임 콘솔 등의 수요 증가로 인해 수요가 증가할 것으로 예상됩니다.\n\n반면, PC DRAM의 수요는 다소 감소할 것으로 예상됩니다. 이는 PC 시장의 성장이 둔화되고 있기 때문입니다.\n\nDRAM 공급 측면에서는, 주요 DRAM 제조업체들이 생산량을 조절하고 있기 때문에 공급 과잉이 발생할 가능성은 낮을 것으로 예상됩니다.\n\nSK하이닉스는 DRAM 시장에서 높은 점유율을 차지하고 있으며, 안정적인 수요와 공급을 바탕으로 꾸준한 성장을 이룰 것으로 예상됩니다.'

In [ ]:
chain.invoke("sk하이닉스의 주가 전망 알려줘")

"SK하이닉스의 주가 전망은 긍정적입니다. 미래에셋증권은 SK하이닉스에 대한 투자의견을 '매수'로 유지하고, 목표주가를 기존 240,000원에서 260,000원으로 8% 상향했습니다. 이는 예상보다 강한 메모리 가격 전망에 따라 2024년과 2025년 영업이익 추정치를 각각 23.4%, 62.5% 상향 조정했기 때문입니다.\n\n특히, 2025년 DRAM 수요 비트그로스(Bit growth)는 +23.0%로 전망되고 있으며, 경기 회복에 대한 명확한 전망이 어려운 상황에서 올해 비트그로스(+17.1%)보다 상승폭이 확대되는 부분이 고무적입니다. 수요 증가의 주요 동력원은 컨벤셔널 서버 수요 회복과 HBM 수요 강세 지속입니다. \n\n더불어, 내년 DRAM 업계의 평균 Wafer capa 증가율은 14.5%에 그치며 수요 성장률보다 낮을 것으로 전망되어, 상기 수급 동향에 따라 2025년 DRAM의 공급초과율은 -9.3%로 올해(-5.1%)보다 타이트한 수급이 심화될 것으로 보입니다. 이에 따라 2025년 DRAM 예상 가격은 올해에 이어 +40%대의 상승세가 지속될 것으로 전망됩니다."

In [ ]:
chain.invoke("sk하이닉스와 관련하여 25년 시장 상황 알려줘")

'25년 DRAM 수요 B/G(Bit growth)는 +23.0%로 전망되고 있습니다. 경기 회복에 대한 명확한 전망이 어려운 상황에서 올해 B/G(+17.1%)보다 상승폭이 확대되는 부분이 고무적이며, 수요 증가의 주요 동력원은 컨벤셔널 서버 수요 회복과 HBM 수요 강세 지속으로 보고 있습니다. \n\n더불어 컨벤셔널 서버의 대대적인 교체주기 도래에 따라 내년 서버 출하량은 10년래 최대 성장률(+4.3%)을 보일 것으로 전망됩니다. 이는 글로벌 빅테크의 17 ~ 18년 대규모 투자 사이클 이후 서버 감가상각 내용연수가 3년 → 6년으로 연장되며 미교체 서버가 누적되어 온 결과로 추정됩니다.\n\n또한 고객사의 Blackwell 플랫폼 본격 판매와 이후 Rubin 플랫폼으로 이어지며 가속기의 HBM 탑재량 증가율이 +85.2%로 올해의 성장률(+66.5%)을 초과할 것으로 기대하고 있습니다.'

In [ ]:
chain.invoke("sk하이닉스의 실적 전망 알려줘")

'SK하이닉스의 2024년 매출액은 68,338십억원, 영업이익은 24,258십억원으로 전망되며, 2025년 매출액은 98,611십억원, 영업이익은 39,606십억원으로 예상됩니다. 이는 메모리 가격 전망이 예상보다 강하기 때문이며, 2024년과 2025년 영업이익 추정치를 각각 23.4%, 62.5% 상향 조정했습니다. \n\n또한, 25년 DRAM 수요 비트 성장률은 23.0%로 전망되고 있으며, 경기 회복에 대한 명확한 전망이 어려운 상황에서도 올해의 비트 성장률인 17.1%보다 상승폭이 확대될 것으로 보입니다. 수요 증가의 주요 동력원은 컨벤셔널 서버 수요 회복과 HBM 수요 강세 지속이며, 25F 서버 DRAM 비트 성장률은 25.0%, 그래픽 DRAM 비트 성장률은 85.5%로 모두 올해의 성장률을 초과할 것으로 기대됩니다.'

In [ ]:
chain.invoke("sk하이닉스 DRAM의 Bit shipments에 대해 분기별로 알려줘")

'네, SK하이닉스 DRAM의 분기별 Bit shipments는 다음과 같습니다.\n\n- 1Q23: 12,088 Bil Gb\n- 2Q23: 16,440 Bil Gb\n- 3Q23: 19,892 Bil Gb\n- 4Q23: 20,290 Bil Gb\n- 1Q24: 17,044 Bil Gb\n- 2Q24: 20,623 Bil Gb\n\n더 궁금하신 점이나 다른 질문이 있으시면 언제든지 말씀해주세요.'

In [ ]:
chain.invoke("미래에셋 지속가능경영에 대해 말해줘")

'미래에셋증권은 지속가능한 경영을 위해 다양한 노력을 기울이고 있습니다. \n\n먼저, 내부통제시스템의 객관적인 검증을 위해 외부검증업체인 한국 거래소, 금융정보분석원 등을 통해 주기적으로 검증을 실시하고 있으며, 투자 기업 및 자산에 대한 탄소배출량(Finance Emission)은 PCAF(탄소회계금융협회)에서 정의하는 자산분류기준 및 산정 방법론을 준용하고 있습니다. 미래에셋증권은 2022년말의 금융배출량을 기준으로 각 자산군별 중·장기 목표를 수립하였고, 2023년 11월에 SBTi 검증을 완료하였습니다. \n\n또한, 전자금융기반시설과 공개용 홈페이지 취약점을 점검하여 보안을 강화하고 있으며, APT공격 대응 훈련과 디도스 공격 대응 훈련을 실시하여 임직원의 보안인식을 제고하고 있습니다.\n\n이러한 노력을 통해 미래에셋증권은 지속가능한 경영을 추구하고, 고객과 사회에 더 나은 가치를 제공하기 위해 노력하고 있습니다.'

In [ ]:
chain.invoke("퇴직연금에 대해 말해줘")

'네. 퇴직연금은 근로자의 노후생활을 보장하기 위해 회사가 근로자에게 지급해야 할 퇴직급여를 회사가 아닌 금융회사(퇴직연금사업자)에 맡기고 기업 또는 근로자의 지시에 따라 운용하여 근로자 퇴직 시 일시금 또는 연금으로 지급하는 제도입니다.\n\n퇴직연금은 확정급여형(DB), 확정기여형(DC), 개인형퇴직연금(IRP) 등 세 가지가 있습니다.\n\n- 확정급여형(DB): 근로자가 받을 퇴직급여가 사전에 확정되며, 회사가 적립금을 운용하고 그 결과에 따라 회사에 책임이 있습니다.\n- 확정기여형(DC): 회사가 매년 근로자 연간임금의 1/12 이상을 부담금으로 납부하고, 근로자가 적립금을 운용합니다. 근로자의 적립금 운용성과에 따라 퇴직급여가 변동될 수 있으며, 회사는 적립금 운용에 대해 책임을 지지 않습니다.\n- 개인형퇴직연금(IRP): 근로자가 이직하거나 퇴직할 때 받은 퇴직급여를 적립·운용하기 위한 퇴직연금제도입니다.\n\n퇴직연금에 대한 자세한 내용은 미래에셋증권 홈페이지에서 확인하실 수 있습니다.'

# 프롬프트와 메모리 반영

In [ ]:
# PromptTemplate 설정
# 사용자가 입력한 정보를 받아오는 함수
def create_prompt(user_info):
    template = """
    당신은 미래에셋증권에서 서비스되는 개인 금융비서입니다.

    내 정보는 다음과 같습니다.

    개인 정보
    나이: {age}세
    성별: {gender}
    직업: {job} (근무 경력 {job_experience}년)
    가족 구성: {family_members}명 (자녀 {children_count}명 포함)

    소득
    연 근로소득: {annual_income}만원
    연 불로소득: {passive_income}만원 (주식 배당금)

    자산
    총 자산: {total_assets}만원
    주택 (아파트): {house_value}만원 (주택담보대출: {mortgage}만원)
    현금: {cash}만원
    미국 S&P ETF: {etf}만원

    내 정보를 토대로 답변하십시오.
    답을 모르면 그냥 모른다고 대답하십시오.
    다음과 같은 맥락과 채팅히스토리를 사용하여 마지막 질문에 대답하십시오.

    맥락: {context}
    채팅히스토리: {chat_history}
    질문: {question}
    도움이 되는 답변:
    """

    # 사용자 정보를 프롬프트에 삽입
    prompt = template.format(
        age=user_info['age'],
        gender=user_info['gender'],
        job=user_info['job'],
        job_experience=user_info['job_experience'],
        family_members=user_info['family_members'],
        children_count=user_info['children_count'],
        annual_income=user_info['annual_income'],
        passive_income=user_info['passive_income'],
        total_assets=user_info['total_assets'],
        house_value=user_info['house_value'],
        mortgage=user_info['mortgage'],
        cash=user_info['cash'],
        etf=user_info['etf'],
        context='{context}',
        chat_history='{chat_history}',
        question='{question}'
    )

    return prompt

In [ ]:
# 사용자의 개인정보 예시
user_info = {
    'age': 48,
    'gender': '남성',
    'job': '회사원 (중견기업)',
    'job_experience': 22,
    'family_members': 4,
    'children_count': 2,
    'annual_income': 11200,
    'passive_income': 300,
    'total_assets': 103000,
    'house_value': 83000,
    'mortgage': 21000,
    'cash': 10000,
    'etf': 10000
}


# 프롬프트 생성
dynamic_prompt = create_prompt(user_info)

# PromptTemplate 설정
rag_prompt_custom = PromptTemplate.from_template(dynamic_prompt)

In [ ]:
# RAG chain 설정
rag_chain = {"context": itemgetter("question") | retriever, "question": itemgetter("question"), "chat_history": itemgetter("chat_history")} | rag_prompt_custom | llm

In [ ]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 세션 기록을 확인하는 함수
def print_session_history(session_ids):
    history = get_session_history(session_ids)
    if history:
        print(f"[세션 기록 - 세션ID {session_ids}]:")
        for message in history.messages:
            print(f"내용: {message.content}")
    else:
        print(f"[세션 기록 없음] 세션ID: {session_ids}")


# 세션 기록을 초기화하는 함수
def clear_session_history(session_ids):
    if session_ids in store:
        del store[session_ids]
        print(f"[세션 기록 초기화 완료] 세션ID: {session_ids}")
    else:
        print(f"[세션 기록 없음] 세션ID: {session_ids}")

In [ ]:
# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

## 질문 예시

In [ ]:
clear_session_history('기남')


[세션 기록 초기화 완료] 세션ID: 기남


In [ ]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "최근 유망한 주식 섹터에 대해 키워드만 제시해줘"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "기남"}},
)

[대화 세션ID]: 기남


'최근 유망한 주식 섹터로는 다음과 같은 키워드가 있습니다.\n\n1. 인공지능(AI)\n2. 빅데이터\n3. 전기차\n4. 수소차\n5. 바이오\n6. 인터넷\n7. 게임\n\n해당 섹터에 속한 기업들의 주식을 분석하여 투자를 결정하는 것이 좋습니다. 단, 주식 시장은 변동성이 있으므로 투자 전에 충분한 조사와 분석이 필요합니다.'

In [ ]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "그것들을 제외하고 하나만 더 제시해줘. 이번에는 설명도 함께 제공해줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "기남"}},
)

[대화 세션ID]: 기남


"최근 유망한 주식 섹터 중에서 앞서 제시한 키워드를 제외하고 하나를 더 제시하자면, '우주산업' 섹터를 추천드립니다.\n\n우주산업은 미래 성장 가능성이 매우 높은 분야 중 하나입니다. 우주산업은 크게 위성산업, 우주여행산업, 우주자원개발산업 등으로 나눌 수 있습니다. \n\n- 위성산업은 인공위성을 이용하여 다양한 서비스를 제공하는 산업입니다. 대표적인 예로는 통신위성, 방송위성, 기상위성 등이 있습니다. 최근에는 자율주행차, 드론, IoT 등의 기술 발전으로 인해 위성산업의 중요성이 더욱 커지고 있습니다.\n\n- 우주여행산업은 우주를 여행하는 산업입니다. 아직은 초기 단계이지만, 우주여행에 대한 수요가 증가하면서 우주여행산업의 규모도 점차 커질 것으로 예상됩니다.\n\n- 우주자원개발산업은 우주에서 자원을 개발하는 산업입니다. 대표적인 예로는 달 탐사, 화성 탐사 등이 있습니다. 우주자원개발산업은 인류의 미래를 위한 중요한 산업 중 하나입니다.\n\n우주산업은 아직까지 발전 초기 단계이기 때문에, 투자 시에는 기업의 기술력과 경쟁력, 시장 동향 등을 충분히 파악하고, 장기적인 관점에서 투자하는 것이 좋습니다."

In [ ]:
clear_session_history('기남')


[세션 기록 초기화 완료] 세션ID: 기남


In [ ]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "안녕! 반가워"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "기남"}},
)

[대화 세션ID]: 기남


'안녕하세요! 미래에셋증권에서 서비스되는 개인 금융비서입니다.\n\n먼저, 사용자님의 개인 정보와 자산 현황을 분석하여 최적의 금융 솔루션을 제공해 드리겠습니다. \n\n사용자님은 중견기업에서 22년간 근무하신 48세 남성으로, 연 근로소득은 11,200만원, 불로소득은 300만원(주식 배당금)이며, 총 자산은 103,000만원입니다. 주택담보대출 21,000만원을 제외한 순자산은 82,000만원입니다.  \n\n현재 미국 S&P ETF에 10,000만원을 투자하고 계신데, 이는 안정적인 수익을 기대할 수 있는 좋은 선택입니다. 하지만, 금융 시장은 변동성이 높기 때문에, 자산 분산을 통해 위험을 관리하는 것이 중요합니다. \n\n저는 사용자님의 금융 목표를 달성하기 위해 다양한 금융 상품을 추천하고, 투자 전략을 제시해 드리겠습니다. 또한, 금융 관련 뉴스와 정보를 제공하여, 사용자님이 금융 지식을 향상시키고, 적극적으로 금융 생활을 하실 수 있도록 돕겠습니다.  \n\n궁금하신 점이나 도움이 필요하신 사항이 있으시면 언제든지 말씀해주세요.'

In [ ]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "요즘 sk하이닉스 관심이 있는데 주가 전망이 어때?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "기남"}},
)

[대화 세션ID]: 기남


'SK하이닉스는 메모리 반도체 시장에서 높은 경쟁력을 가진 기업으로, 최근 메모리 가격 상승과 함께 실적 개선이 기대되고 있습니다. \n\n미래에셋증권 리서치센터의 분석에 따르면, SK하이닉스의 2024년 영업이익은 24조 2,580억원으로 전년 대비 215% 증가할 것으로 예상되며, 2025년에는 39조 6,060억원으로 더욱 증가할 것으로 전망됩니다. 이에 따라, 목표주가를 26만원으로 상향 조정하였습니다.\n\n하지만, 주식 시장은 다양한 요인에 의해 변동될 수 있으므로, 투자 전에는 기업의 재무상태와 경영실적, 시장 동향 등을 종합적으로 고려하여 신중하게 판단하시는 것이 좋습니다.'

In [ ]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "그 기업의 주요 제품은 어떤 것이 있어?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "기남"}},
)

[대화 세션ID]: 기남


'SK하이닉스의 주요 제품으로는 DRAM과 NAND가 있으며, 이 두 제품이 전체 매출에서 차지하는 비중은 약 80% 이상입니다. 특히, DRAM은 글로벌 시장에서 점유율 2위를 차지하고 있으며, NAND는 인텔의 낸드 사업부를 인수하여 점유율을 확대하고 있습니다. \n\n해당 기업에 대한 자세한 정보는 미래에셋증권 리서치센터에서 제공하는 자료를 참고하실 수 있습니다.'

In [ ]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "DRAM의 생산 및 판매량과 관련된 지표에 대해 분기별로 알려줄 수 있어?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "기남"}},
)

[대화 세션ID]: 기남


'네. DRAM의 생산 및 판매량과 관련된 지표는 다음과 같은 분기별 지표가 있습니다.\n\n- DRAM Bit shipments (출하량, 백만 기가비트) \n- DRAM ASP (평균판매가격, 달러/기가비트) \n\n위의 지표는 DRAM의 생산 및 판매량을 파악하는 데 중요한 역할을 하며, 이를 통해 메모리 반도체 시장의 동향을 파악할 수 있습니다.\n\n미래에셋증권 리서치센터에서는 위와 같은 지표를 포함한 다양한 자료를 제공하고 있으며, 이를 참고하여 메모리 반도체 시장의 동향을 파악하고, 투자 전략을 수립할 수 있습니다.'

In [ ]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "DRAM Bit shipments의 연도별, 분기별 추정치에 대해 알려줘"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "기남"}},
)

[대화 세션ID]: 기남


'네. DRAM Bit shipments의 연도별, 분기별 추정치는 다음과 같습니다. \n\n- 2023년 3분기: 19,892 백만 기가비트, ASP 0.23 달러/기가비트\n- 2023년 4분기: 20,290 백만 기가비트, ASP 0.27 달러/기가비트\n- 2024년 1분기: 17,044 백만 기가비트, ASP 0.33 달러/기가비트 \n- 2024년 2분기: 20,623 백만 기가비트, ASP 0.38 달러/기가비트\n\n위의 정보는 미래에셋증권 리서치센터에서 제공하는 자료를 참고하여 작성하였으며, 시장 상황에 따라 변동될 수 있습니다.'

# 2. Multi-vector retriever 영구 저장

In [ ]:
class PDFPartitionConfig(BaseModel):
    extract_images_in_pdf: bool = True
    infer_table_structure: bool = True
    chunking_strategy: str = "by_title"
    max_characters: int = 4000
    new_after_n_chars: int = 3800
    combine_text_under_n_chars: int = 2000
    image_output_dir_path: str

def process_pdfs_in_directory(directory: str, config: PDFPartitionConfig) -> Any:
    raw_pdf_elements = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            pdf_elements = partition_pdf(
                filename=file_path,
                extract_images_in_pdf=config.extract_images_in_pdf,
                infer_table_structure=config.infer_table_structure,
                chunking_strategy=config.chunking_strategy,
                max_characters=config.max_characters,
                new_after_n_chars=config.new_after_n_chars,
                combine_text_under_n_chars=config.combine_text_under_n_chars,
                image_output_dir_path=config.image_output_dir_path
            )
            raw_pdf_elements.append((filename, pdf_elements))
    return raw_pdf_elements


In [ ]:
path = db_company_path = "/content/drive/MyDrive/miraeasset/data/company"
config = PDFPartitionConfig(image_output_dir_path=path)

In [ ]:
raw_pdf_elements = process_pdfs_in_directory(path, config)

In [ ]:
all_pdf_elements = []
for _, elements in raw_pdf_elements:
    all_pdf_elements.extend(elements)

In [ ]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in all_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 27,
 "<class 'unstructured.documents.elements.Table'>": 19,
 "<class 'unstructured.documents.elements.TableChunk'>": 2}

In [ ]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in all_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

21
27


# Summaries

In [ ]:
tables = [i.text for i in table_elements]
table_summaries = process_texts_concurrently(tables)

Error: HTTP error: 400 - Bad Request


In [ ]:
texts = [i.text for i in text_elements]
text_summaries = process_texts_concurrently(texts) # 39초

# Add to vectorstore

In [ ]:
db_company_path = "/content/drive/MyDrive/miraeasset/ChromaDB/DB_company"

In [ ]:
import os
os.chdir('/content/drive/MyDrive/miraeasset/code')
from embedding import HyperClovaEmbedding, HyperClovaEmbeddings

In [ ]:
# HyperClovaEmbedding 인스턴스 생성
hyper_clova = HyperClovaEmbedding(
    host='clovastudio.apigw.ntruss.com',
    api_key='your_api_key_here',
    api_key_primary_val='your_primary_api_key_here',
    request_id='your_request_id_here'
)

In [ ]:
# 임베딩 생성 요청
try:
    text = "input text"
    embedding = hyper_clova.execute(text)
    print(f"Embedding: {embedding}")
except ValueError as e:
    print(e)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

# 재시도 간격을 늘리고, 최대 재시도 횟수를 증가시킵니다.
# 요청 실패 시 백오프(backoff) 전략을 도입하여 재시도 간격을 지수적으로 증가시킵니다.

Embedding: [-0.9326172, 0.69628906, -0.515625, 0.12683105, -0.29736328, -0.55615234, 0.2388916, 0.15234375, 0.40039062, -0.68359375, 0.29541016, -0.047424316, 0.36669922, -0.81689453, -0.2619629, -0.34375, -0.16601562, 0.36743164, 0.17993164, -0.2705078, -0.80078125, -0.24499512, -0.08093262, 0.34033203, 0.47387695, 0.9682617, 0.2866211, -0.42407227, -0.58154297, -0.20178223, 0.87646484, 0.0181427, -0.7939453, -2.0625, 0.31201172, -0.11657715, -0.12573242, -0.3149414, -1.5751953, 0.27807617, 0.087524414, -0.59716797, 0.55615234, -0.022842407, -0.27807617, -0.081970215, 1.0087891, -0.39575195, -0.7050781, -0.5683594, 0.054107666, 0.52685547, 1.8037109, -0.1784668, -0.31225586, -0.5307617, -0.2388916, 0.16149902, -1.6572266, -0.07635498, -0.2788086, -0.54296875, -0.48706055, -0.27929688, -0.21374512, 1.7119141, 0.91308594, 0.13464355, -0.703125, -0.67089844, -0.33496094, 0.7866211, -0.3413086, -0.08496094, -1.2548828, 1.1201172, 0.6401367, -0.40893555, -0.41479492, 0.6435547, 0.6928711, 

In [ ]:
embedding_func = HyperClovaEmbeddings(hyper_clova)

In [ ]:
db_company_path = "/content/drive/MyDrive/miraeasset/ChromaDB/DB_company"
# The vectorstore to use to index the child chunks
db_company = Chroma(collection_name="summaries",
                     embedding_function=embedding_func,
                     persist_directory=db_company_path) # db_company

In [ ]:
# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=db_company,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

In [ ]:
retriever.vectorstore.similarity_search_with_score("sk하이닉스에 대해 말해줘")

[(Document(metadata={'doc_id': '0348e148-749f-4663-9939-7b912ad18c68'}, page_content='- SK하이닉스의 밸류에이션을 설명하고 있음'),
  393.7416076084247),
 (Document(metadata={'doc_id': '1abeb071-6eb9-48f0-8c75-70099f91bba7'}, page_content='- SK하이닉스 주요 제품별 추정치를 보고 있음'),
  539.1353932568777),
 (Document(metadata={'doc_id': '40238298-94c1-432b-ad80-4b5de1a2e2d2'}, page_content='- SK하이닉스 수익추정 변경 내역을 보고 있음'),
  547.6118496655075),
 (Document(metadata={'doc_id': '25b3063f-98dc-471f-b9f3-8e9c0688160e'}, page_content='- SK하이닉스의 예상 포괄손익계산서와 예상 현금흐름표를 요약함'),
  550.4959065737272)]

# vectorstore 및 docstore 저장

In [ ]:
# vectorstore
db_company.persist()

In [ ]:
import pickle

# docstore 저장
with open("/content/drive/MyDrive/miraeasset/ChromaDB/DB_company/db_company_docstore.pkl", "wb") as f:
    pickle.dump(store, f)

# vectorstore 및 docstore 로드

In [ ]:
# Chroma vectorstore 로드
db_company_path = "/content/drive/MyDrive/miraeasset/ChromaDB/DB_company"

db_company_new = Chroma(collection_name="summaries",
                     embedding_function=embedding_func,
                     persist_directory=db_company_path)

# InMemoryStore 로드
with open("/content/drive/MyDrive/miraeasset/ChromaDB/DB_company/db_company_docstore.pkl", "rb") as f:
    store_new = pickle.load(f)


# MultiVectorRetriever 초기화
id_key = "doc_id"
retriever = MultiVectorRetriever(
    vectorstore=db_company_new,
    docstore=store_new,
    id_key=id_key,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
retriever.vectorstore.similarity_search_with_score("sk하이닉스에 대해 말해줘")

[(Document(metadata={'doc_id': '0348e148-749f-4663-9939-7b912ad18c68'}, page_content='- SK하이닉스의 밸류에이션을 설명하고 있음'),
  393.7416075933443),
 (Document(metadata={'doc_id': '1abeb071-6eb9-48f0-8c75-70099f91bba7'}, page_content='- SK하이닉스 주요 제품별 추정치를 보고 있음'),
  539.1353955600032),
 (Document(metadata={'doc_id': '40238298-94c1-432b-ad80-4b5de1a2e2d2'}, page_content='- SK하이닉스 수익추정 변경 내역을 보고 있음'),
  547.6118493299783),
 (Document(metadata={'doc_id': '25b3063f-98dc-471f-b9f3-8e9c0688160e'}, page_content='- SK하이닉스의 예상 포괄손익계산서와 예상 현금흐름표를 요약함'),
  550.4959054122132)]

In [ ]:
retriever.vectorstore.similarity_search_with_score("DRAM")

[(Document(metadata={'doc_id': 'd314c5e0-533b-40ce-89d9-8c0c7d3c625b'}, page_content='- DRAM 수요 추이 및 전망을 보고 있음'),
  411.81610819587746),
 (Document(metadata={'doc_id': '33860644-f2d6-4eb6-8c32-8d307ceeaa72'}, page_content='- DRAM의 종류를 설명하고 있음\n'),
  426.21148992363385),
 (Document(metadata={'doc_id': 'b2d93855-dcd7-4009-a44f-0a3a34f00cc0'}, page_content='- DRAM 공급 추이 및 전망에 대해 이야기하고 있음'),
  432.65081620352925),
 (Document(metadata={'doc_id': 'cc2f4390-3df1-4b37-bc41-cd3aa1892629'}, page_content='\n'),
  453.6719175203565)]